In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo-preview")


Let's create a vector Database

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb, os

embedding = OpenAIEmbeddings()

raw_documents = DirectoryLoader("./docs",glob="**/*.txt", show_progress=False).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(raw_documents)

chroma_directory = "chromadb"
if os.path.exists(chroma_directory):
    db = Chroma(embedding_function=embedding, persist_directory=chroma_directory)
else:
    db = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory=chroma_directory)

In [4]:
from langchain.chains import RetrievalQA
# RAG prompt
from langchain import hub

# Loads the latest version
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=db.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

Define a tool to fetch crime record

In [5]:
from langchain.agents import Tool, AgentExecutor, ZeroShotAgent
import requests
import json

def fetch_crime_records(json_request: str):
    url = 'http://localhost:8080/'
    endpoint = 'search_crime_records'
    arguments = json.loads(json_request)
    location = arguments.get('location',None)
    crime_type = arguments.get('crime_type',None)
    r = requests.get(url + endpoint, params={'location': location, 'crime_type': crime_type})
    return r.content

fetch_crime_records_tool = Tool(
    name="Get Crime record by location or crime type - tool/API",
    func=fetch_crime_records,
    description="""Fetches the crime records based on location or crime type or both using API endpoints,
    input should be a JSON string with location and crime_type as keys""",
    handle_tool_error=True
)





In [6]:
from langchain.chains import LLMChain
prompt = ZeroShotAgent.create_prompt(
    tools=[fetch_crime_records_tool],
    prefix="You are a helpful assistant that helps people get crime records. {instruction}\n\
            You have this chat history: {chat_history}\n\n",
    format_instructions="""use the following format:\n
    Question: the input question you must answer.
    Thought: you should always think about which tool to use.
    Action: the action to take, should always be only one of the tools '{tool_names}'
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: Summarize the observation which is the final answer to the original input question
    """,
    suffix="Begin:\nQuestion: {input}\nThought: {agent_scratchpad}",
    input_variables=['input', 'instruction'],
)
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

In [7]:
agent = ZeroShotAgent(
    llm_chain = llm_chain,
    tools = [fetch_crime_records_tool],
    verbose=True
)


c:\Users\manoj\Documents\MANOJ\Github\Ask-API\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [8]:
from langchain.memory import ConversationTokenBufferMemory

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=[fetch_crime_records_tool],
    handle_parsing_errors=True,
    memory=ConversationTokenBufferMemory(memory_key="chat_history", max_token_limit=500,llm=llm),
    max_iterations=5,
    verbose=True
)

In [11]:
question = "Get crime records on crime type Homicide which happened near location 'Susan Spring'?"

In [12]:
instruction_from_llm = qa_chain.invoke({"query": question})['result']
agent_executor.run({"input": question,"instruction": instruction_from_llm})



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that helps people get crime records. I don't have the ability to directly access or retrieve real-time data, including crime records from external databases or APIs. To get crime records on the crime type Homicide near the location 'Susan Spring,' you would need to utilize a relevant database or API that provides access to crime records using the specified parameters. This process typically involves searching with parameters like the location ('Susan Spring') and crime type (Homicide) to find matching crime records.
            You have this chat history: Human: I don't have access to real-time data or specific databases to retrieve crime records for an incident near 'Susan Spring.' To find information on a homicide that occurred near 'Susan Spring,' you would need to utilize a crime records database or API with the ability to search by location and crime type

"A homicide occurred near 'Susan Spring' with the following details: Crime Record ID is 4ad2c089-979c-406a-8ca3-17921d46a200, located at 974 Susan Spring, Port Joshuaborough, VT 12878. The suspects involved are listed as Alison Jackson. The victim of this crime was Linda Gentry, and Anthony Dorsey was a witness. The description of the incident includes various elements suggesting a complex scenario, with mentions of government style, window institution attack, and specifics about the crime scene. The incident occurred on 2024-02-03 at 09:39:31."